In [3]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

검색할 키워드를 입력해주세요: 경제

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력): 1



크롤링할 시작 페이지:  1 페이지



크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력): 20



크롤링할 종료 페이지:  20 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=경제&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=경제&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=경제&start=21', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=경제&start=31', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=경제&start=41', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=경제&start=51', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=경제&start=61', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=경제&start=71', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=경제&start=81', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=경제&start=91', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=경제&start=101', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=경제&start=11

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 73/73 [00:09<00:00,  7.39it/s]


검색된 기사 갯수: 총  200 개

[뉴스 제목]
['미국 전체가 \'들썩\'…"경제효과 8조원 추정"', '송미령 "농축산물 물가 엄중 인식…조속히 안정시킬 것"', '미중, 불공정 관행 등 양국 경제 현안 회담 수주 내 별도로 연다', '경북도, 드라마·영화 촬영 유치해 지역경제 활성화 도모', '운명의 총선…3대 개혁 완수 vs 경제 민주화 개헌', '평택시민지역경제살리기비대위, 한무경 후보 지지선언', '"경제동맹 실현" 부울경 초광역권 시행계획 수립', '17년 만에 금리 인상, 日 경제 부활 신호탄 될까\xa0', '\'규제완화\' 외쳤지만, 규제 계속 늘어…"경제활동 저해"', "韓 '조국·이재명' vs 李 '대통령·경제'", '인천시 역사 문화 경제 시설 무료 견학·체험 프로그램 운영', '"경제동맹 실현" 부울경 초광역권 시행계획 수립', '17년 만에 금리 인상, 日 경제 부활 신호탄 될까\xa0', '\'규제완화\' 외쳤지만, 규제 계속 늘어…"경제활동 저해"', '"안 빼고 나가면 부기가 그대로 살 돼"…산모는 마사지까지 긁었다 [헛다리경제]', '옐런, 中경제중심지서 \'공급과잉\' 지적…"감당 규모 넘어"(종합)', '[다음주 경제] 기준금리 10연속 동결되나', "韓 '조국·이재명' vs 李 '대통령·경제'", 'SPC 초유의 경영 공백…글로벌 진출 난항, 서민 경제 타격 우려도', '금천구 ‘학교로 찾아가는 공동체경제 아카데미’ 운영', 'D-3 막판 화력전…野 "경제 무너져"↔與 "野오만·부도덕에 분노"', '[우석훈의 경제수다방]정치적 전환기 공무원의 역할', '고용시장 석 달 연속 훈풍불까…청년 취업률도 주목[경제전망대]', '[통통 지역경제] 수소·이차전지산업 메카 이끄는 울산경자청', "美·中 '균형잡힌 경제성장' 회담 개최 합의…'과잉생산' 의제엔 온도차(종합)", '‘모디의 인도’ 10년간...인구 1위, 증시 4위, 경제규모 5위 [헬로인디아]', '[설왕설래] ‘답정너’ 최저임금 결정구조', '고물가·가계부채·부동

In [4]:
news_df

,date,title,link,content
0,2024-04-07 07:40:28,"미국 전체가 '들썩'…""경제효과 8조원 추정""",https://n.news.naver.com/mnews/article/215/000...,[\n\n\n\n\n7년 만에 관측되는 개기일식을 앞두고 미국 전역이 들썩이는 가운...
1,2024-04-07 16:30:00,"송미령 ""농축산물 물가 엄중 인식…조속히 안정시킬 것""",https://n.news.naver.com/mnews/article/003/001...,"[\n농식장관, 전주 완주로컬푸드직매장 방문""로컬푸드 직매장 등 지원 혜택 누리길""..."
2,2024-04-07 00:42:04,"미중, 불공정 관행 등 양국 경제 현안 회담 수주 내 별도로 연다",https://n.news.naver.com/mnews/article/014/000...,"[\n옐런 미 재무, 과잉 생산 등 거시경제 불균형 논의 \n\n\n\n중국을 방..."
3,2024-04-07 09:16:40,"경북도, 드라마·영화 촬영 유치해 지역경제 활성화 도모",https://n.news.naver.com/mnews/article/001/001...,"[\n\n\n\n\n경북도, 상주시, ㈜앤피오엔터테인먼트 업무 협약[경북도 제공. ..."
4,2024-04-07 18:28:11,운명의 총선…3대 개혁 완수 vs 경제 민주화 개헌,https://n.news.naver.com/mnews/article/015/000...,[\n4대 시나리오별 정국 변화野 과반 땐 '입법 공백' 재연\n\n\n\n22대 ...
...,...,...,...,...
64,2024-04-05 06:01:01,尹 정부 출범 후 10兆 투자 몰린 새만금… “경제 효과는 27兆”,https://n.news.naver.com/mnews/article/366/000...,"[\n투자 유치, 2013년 개청 후 9년간 실적의 7배 전북특별자치도..."
65,2024-04-04 15:11:13,"남원시, 백종원 더본코리아와 지역경제 활성화 '맞손'",https://n.news.naver.com/mnews/article/003/001...,[\n\n\n\n\n4일 남원시청에서 열린 남원시와 더본코리아의 '지역경제 활성화를...
66,2024-04-07 17:17:03,역대 최대 대미 흑자…트럼프 당선 땐 ‘부메랑’될라,https://n.news.naver.com/mnews/article/028/000...,"[\n한국, 미국의 8대 적자국 올라트럼프, 경제 압박 나설 가능성“가스 등 수입 ..."
67,2024-04-06 16:06:10,[포토] 中 경제부총리와 악수하는 옐런 美재무장관,https://n.news.naver.com/mnews/article/421/000...,[\n\n\n\n\n(광저우 로이터=뉴스1) 김성식 기자 = 재닛 옐런 미국 재무장...


In [6]:
news_df.to_json('economicNews.json', orient='records', indent=4,force_ascii=False)